## Homework 1

## <em>Numerical Integration and ODE/PDEs</em>
<br>
This notebook is arranged in cells. Texts are usually written in the markdown cells, and here you can use html tags (make it bold, italic, colored, etc). You can double click on this cell to see the formatting.<br>
<br>
The ellipsis (...) are provided where you are expected to write your solution but feel free to change the template (not over much) in case this style is not to your taste. <br>
<br>
<em>Hit "Shift-Enter" on a code cell to evaluate it.  Double click a Markdown cell to edit. </em>

***
### Link Okpy

In [ ]:
from client.api.notebook import Notebook
ok = Notebook('hw1_288.ok')
_ = ok.auth(inline = True)

***

### Imports

In [2]:
import numpy as np
#Import in-built functions for different integration techniques
#For reference: https://docs.scipy.org/doc/scipy/reference/integrate.html
from scipy.integrate import quad, fixed_quad, romberg, dblquad
#For plotting
import matplotlib.pyplot as plt
%matplotlib inline

#### Problem 1 - Gaussian Quadrature

<span style="color:blue"> Define the manual function for Gaussian quadrature integration. </span>

Hint: First define a function which calculates N positions and weights and them use them for integratation between points a and b. 


In [3]:
def gaussxw(N):
    '''Calculate 'N' position and weights for Gaussian quadrature integration
    Returns a tuple of 2 arrays, the first array is the position of points and second
    array is the corresponding weights.
    '''
    a = np.linspace(3, 4*N -1, N)/(4*N+2)
    x = np.cos(np.pi*a + 1/(8*N*N*np.tan(a)))
    eps = 1e-15
    delta = 1.
    #calc roots
    while delta>eps:
        p0 = np.ones(N)
        p1 = np.copy(x)
        for k in range(1, N):
            p0, p1 = p1, ((2*k +1)*x*p1 -k*p0)/(k+1)
        dp = (N+1)*(p0 -x*p1)/(1-x**2)
        dx = p1/dp
        x -= dx
        delta = max(abs(dx))
    
    #calc weights
    w = 2*(N+1)**2/(N*N*(1 - x**2)*dp**2)
    
    return x, w

def gaussxwab(N, a, b):
    '''Calculate 'N' position and weights for Gaussian quadrature integration
    between 'a' and 'b'
    Returns a tuple of 2 arrays, the first array is the position of points and second
    array is the corresponding weights.
    '''
    x, w = gaussxw(N)
    return 0.5*(b-a)*x + 0.5*(b+a), 0.5*(b-a)*w

#### Problem 2 - Harmonic Oscillator

The total energy of a harmonic oscillator is given by <br>
$$ E = \frac{1}{2}m \left(\frac{dx}{dt}\right)^2 + V(x) $$
Assuming that the potential $V(x)$ is symmetric about $x=0$ and the amplitude of the oscillator is $a$. Then the equation for the time period is given by <br>
$$ T = \sqrt{8m} \int_0^a \frac{dx}{\sqrt{V(a) - V(x)}} $$

<span style="color:blue"> Q1. Suppose the potential is $V(x) = x^4$ and mass of the particle $m=1$, write a function that calculates the period for a given amplitude. </span>



In [17]:
def V(x):
    return x**4

def timep(x, a):
    'Define the function that needs to be integrated (integrand) to calculate time period'
    m = 1
    return np.sqrt(8*m)/np.sqrt(V(a)-V(x))

def calculate_period(N, amplitude):
    positions, weights = gaussxwab(N, 0, amplitude)
    
    result = 0
    for i in range(len(positions)):
        result += weights[i]*timep(positions[i], amplitude)
        
    return result

calculate_period(10000, 2)

1.854013113288542

<span style="color:blue"> Q2. Let $a = 2$. Use inbuilt 'fixed_quad' (https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.fixed_quad.html) function to calculate the time period for different values of 'N' (number of integration points). Calculate the error in the integral by estimating the difference for 'N' & '2N'. Approximately, at what 'N' is the absolute error less than $10^{-4}$ for 'a = 2'? </span>

<span style="color:blue"> Q3. Use inbuilt 'quad' (https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html) function that returns an error estimate and compare your answer for 'a = 2' (quad uses a more advanced integration technique) </span>

In [18]:
a = 2
N = 10000

fixed_quad1 = fixed_quad(timep, 0, a, args=(a,), n=N)
fixed_quad2 = fixed_quad(timep, 0, a, args=(a,), n=2*N)

print('Using fixed_quad: ')
print('\nFor n = %d, the time period is %0.3f, with error = %0.3e'%(N, fixed_quad1[0], abs(fixed_quad2[0] - fixed_quad1[0])))

tquad = quad(timep, 0, a, args=(a,))
print('Using quad: ')
print('Inbuilt Gaussian Quadrature gives time period = ', tquad[0], ' with error = ', tquad[1])

Using fixed_quad: 

For n = 10000, the time period is 1.854, with error = 3.079e-05
Using quad: 
Inbuilt Gaussian Quadrature gives time period =  1.8540746773017045  with error =  2.0068946504636642e-10


Use the inbuilt romberg function (https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.romberg.html) to use Romberg integration. 

<span style="color:blue"> Q4. A simplistic usage with romberg(func, 0, a), where a is the amplitude, will probably give error or 'nan'. Why? </span>



This is because it diverges at the endpoint a.

Assume that we can tolerate the uncertainitiy of $10^{-5}$ in the position.

<span style="color:blue"> Q5. Show and output of 'keyword' show = True for 'a = 2'. Use this to estimate error for divmax = 10. Show your calculation and compare it with the python warning.</span>

In [20]:
romb = romberg(timep, 0, a-10**-5, args=(a,), show=True, divmax=10)

Romberg integration of <function vectorize1.<locals>.vfunc at 0x000001A647D9E730> from [0, 1.99999]

 Steps  StepSize   Results
     1  1.999990 158.820789 
     2  0.999995 80.140687 53.913986 
     4  0.499997 40.852201 27.756040 26.012177 
     8  0.249999 21.248278 14.713636 13.844143 13.650999 
    16  0.124999 11.476276  8.218943  7.785963  7.689802  7.666424 
    32  0.062500  6.611947  4.990504  4.775274  4.727486  4.715869  4.712985 
    64  0.031250  4.195270  3.389711  3.282992  3.259305  3.253547  3.252118  3.251761 
   128  0.015625  2.997937  2.598826  2.546101  2.534404  2.531561  2.530855  2.530679  2.530635 
   256  0.007812  2.407066  2.210109  2.184195  2.178450  2.177054  2.176708  2.176621  2.176600  2.176594 
   512  0.003906  2.117141  2.020499  2.007858  2.005059  2.004379  2.004210  2.004168  2.004157  2.004155  2.004154 
  1024  0.001953  1.976062  1.929036  1.922939  1.921591  1.921263  1.921182  1.921162  1.921157  1.921155  1.921155  1.921155 

The final re

C:\Users\Rohit Penumarti\Anaconda3\lib\site-packages\scipy\integrate\quadrature.py:784: AccuracyWarning: divmax (10) exceeded. Latest difference = 8.299910e-02
  AccuracyWarning)


In [22]:
result_est = 1.921155
result_est1 = 2.004154
error = (1/(4**10-1))*abs(result_est-result_est1)
print(error)

7.915409007462534e-08


<span style="color:blue"> Q6. Change divmax to change the number of divisions. How does the accuracy change on going from 10 to 15 divisions. </span>


In [21]:
romb1 = romberg(timep, 0, a-10**-5, args=(a,), show=True, divmax=15)

Romberg integration of <function vectorize1.<locals>.vfunc at 0x000001A647D9EEA0> from [0, 1.99999]

 Steps  StepSize   Results
     1  1.999990 158.820789 
     2  0.999995 80.140687 53.913986 
     4  0.499997 40.852201 27.756040 26.012177 
     8  0.249999 21.248278 14.713636 13.844143 13.650999 
    16  0.124999 11.476276  8.218943  7.785963  7.689802  7.666424 
    32  0.062500  6.611947  4.990504  4.775274  4.727486  4.715869  4.712985 
    64  0.031250  4.195270  3.389711  3.282992  3.259305  3.253547  3.252118  3.251761 
   128  0.015625  2.997937  2.598826  2.546101  2.534404  2.531561  2.530855  2.530679  2.530635 
   256  0.007812  2.407066  2.210109  2.184195  2.178450  2.177054  2.176708  2.176621  2.176600  2.176594 
   512  0.003906  2.117141  2.020499  2.007858  2.005059  2.004379  2.004210  2.004168  2.004157  2.004155  2.004154 
  1024  0.001953  1.976062  1.929036  1.922939  1.921591  1.921263  1.921182  1.921162  1.921157  1.921155  1.921155  1.921155 
  2048  0.000

C:\Users\Rohit Penumarti\Anaconda3\lib\site-packages\scipy\integrate\quadrature.py:784: AccuracyWarning: divmax (15) exceeded. Latest difference = 1.213942e-03
  AccuracyWarning)


<span style="color:blue"> Q7. Use the function to make a graph of the period for amplitude ranging from a=0 to a=2.  </span>

In [ ]:
a_vals = np.linspace(0, 2, 100)

***

#### Problem 3 - Black Body Radiation

The total rate at which energy is radiated by a black body per unit area over all frequencies is 

$$ W = \frac{2 \pi k_B^4T^4}{c^2 {h}^3} \int _0^\infty \frac{x^3}{e^x -1} dx $$

<span style="color:blue"> Q1. Write a function to to evaluate the integral in this expression. You will need to change the variables to go from an infinite range to a finite range. What is the change of variable and new functional form? </span>


#### Hint

The variable to go from range 0 to $\infty$ to a finite range of is 

$$ z = \frac{x}{1+x} $$
or equivalently
$$ x = \frac{z}{1-z} $$

In [ ]:
...

According to Stefan's law, the total energy given off by a black-body per unit area per second is given by 
$$ W = \sigma T^4 $$

<span style="color:blue"> Q2. Use the integral to calculate the value of Stefan Boltzmann constant $\sigma$. Use 'fixed_quad' function to do the integral. How accurate you think the answer is? </span>


In [ ]:
...


<span style="color:blue"> Q3. Inbuilt 'quad' function can support an infinite range for integration. Write another function to do the integration from 0 to $\infty$ and compare your answer. </span>

In [ ]:
...

***

#### Problem 4 - Gravitational Pull of Uniform Sheet

The gravitational force due to a plate felt by a point mass of 1 kg a distnace $z$ from the center of the square in the direction perpendicular to the sheet is given by 
$$ F_z = G \sigma z \int \int_{-L/2}^{L/2} \frac{dx dy}{(x^2 + y^2 + z^2)^{3/2}}$$
where $G = 6.674 \times 10 ^{-11} m^3 kg^{-1} s^{-2}$ and $\sigma$ is the mass per unit area. <br>

<span style="color:blue"> Q1. Write a program to calcualte and plot the force as a function of $z$ from $z=0$ to $z=10$ for a sheet of 10 metric tonnes and the sheet is $10\ m$ on side. Use Gaussian quadrature for the double integral. Though there is a 'dblquad' routine in python, we will make use of the manual functions defined in Problem 1. Study how the number of integration points 'N' affects the integral here. (Try N = 500 and N = 1000 and compare results: i.e. plot $F_z$ vs. $z$)</span>

In [ ]:
...

##### Hint: 
The loop is supposed to calculate the double integral. <br>
To fill in the for loop, study what is returned by the 'gaussxwab' function and how do you use it to evaluate the integral. The easiest thing that can possibly be done is write a triple for loop. <br>
However for loops are quite slow in python. You should be able to reduce it to a double loop by using inbuilt functions on numpy array. Take inspiration from here https://docs.scipy.org/doc/numpy/reference/routines.math.html <br>
The next way to avoid loops in python is use [broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html) and/or use of numpy.einsum. Though you are not required to use it here, its nevertheless a handy thing to know about and this problem is one of the simplest cases where you can use it. You will not need to add any new loops inside the one given (though you may need to declare new variables), and it should be faster than both 2 and 3 for loops.

In [ ]:
...

***

#### Problem 5 - Planetary Orbit Integration

One of the great achievements in the history of science was the discovery of the laws of J. Kepler, based on many precise measurements of the positions of Mars by Tycho Brahe and himself. The planets move in elliptic orbits with the sun at one of the foci (Kepler’s first law).

Newton (Principia 1687) then explained this motion by his general law of gravitational attraction (proportional to $1/r^2$) and the relation between forces and acceleration. This then opened the way for treating arbitrary celestial motions by solving differential equations.

![alt text](elliptical_orbit.png "Title")

Consider the following two-body problem, wherein a single planet orbits around a large star. Stellar mass is much larger than planetary mass, so we choose the star as the center of our coordinate system. Now, consider the planet's two-dimensional elliptical orbit around the star. The position of the planet is given by the coordinates $q = (q_1, q_2)$, with the planet's velocity given by $p = \dot{q}$.

Newton's laws, with a suitable normalization, yield the following ordinary differential equations:

$$ \ddot{q_1} = -\frac{q_1}{(q_1^2 + q_2^2)^{3/2}},\ \ \ \ddot{q_2} = -\frac{q_2}{(q_1^2 + q_2^2)^{3/2}}. $$

This is equivalent to a Hamiltonian system with the Hamiltonian:
$$ H(p,q) = \frac{1}{2}(p_1^2 + p_2^2) - \frac{1}{\sqrt{q_1^2 + q_2^2}} $$
$$p_i = \dot{q_i}$$

We will consider the initial position and velocity of the planet to be:
$$ q_1(0) = 1-e,\ \ \ q_2(0) = 0,\ \ \ \dot{q_1}(0) = 0,\ \ \ \dot{q_2}(0) = \sqrt{\frac{1+e}{1-e}} $$

Now determine $q$ as a function of time $t$.

<span style="color:blue"> Q1. Using 400000 steps, use the explicit Euler method (Let $f(q) = \frac{dq}{dt}$. Then, $q(t+\Delta t) = \Delta t \cdot f(q)$ for small $\Delta t$) and plot the orbit of the planet. Assume $e = 0.6$ and integrate to a final time of $T_f = 200$. </span>

In [ ]:
...

<span style="color:blue"> Q2. Using 400000 steps, use the symplectic Euler method</span>

$$ p_{n+1} = p_n - \Delta t H_q(p_{n+1}, q_n) $$
$$ q_{n+1} = q_n + \Delta t H_p(p_{n+1}, q_n) $$
or
$$ q_{n+1} = q_n + \Delta t H_p(p_n, q_{n+1}) $$
$$ p_{n+1} = p_n - \Delta t H_p(p_n, q_{n+1}) $$
where $H_p$ and $H_q$ denote the column vectors of partial derivatives of the Hamiltonian with respect to $p$ and $q$, respectively. i.e. $H_{p_1} = p_1, H_{q_1} = \frac{q_1}{(q_1^2 + q_2^2)^{3/2}}, H_{p_2} = p_2, H_{q_2} = \frac{q_2}{(q_1^2 + q_2^2)^{3/2}}$. 

<span style="color:blue"> Again plot the orbit of the planet. Compare your results in Q1 and Q2 by plotting both solutions in the same figure.</span>

In [ ]:
...

***

## To Submit
Execute the following cell to submit.
If you make changes, execute the cell again to resubmit the final copy of the notebook, they do not get updated automatically.<br>
__We recommend that all the above cells should be executed (their output visible) in the notebook at the time of submission.__ <br>
Only the final submission before the deadline will be graded. 



In [ ]:
_ = ok.submit()
